https://mne.tools/stable/auto_tutorials/preprocessing/15_handling_bad_channels.html

In [10]:
import os
from copy import deepcopy
import numpy as np
import mne
import mne_bids
%matplotlib qt
import matplotlib.pyplot as plt

In [42]:
# Load the data
from mne_bids import (BIDSPath,read_raw_bids)

# path where to save the datasets.
bids_root = "Dataset/ds004147-filtered"
subject_id = '36'


bids_path = BIDSPath(subject=subject_id,task="casinos",
                     datatype='eeg', suffix='eeg',
                     root=bids_root)

# read the file
raw = read_raw_bids(bids_path)
# fix the annotations readin
# ccs_eeg_utils.read_annotations_core(bids_path,raw)

raw.load_data()

Extracting parameters from Dataset\ds004147-filtered\sub-36\eeg\sub-36_task-casinos_eeg.vhdr...
Setting channel info structure...
Reading events from Dataset\ds004147-filtered\sub-36\eeg\sub-36_task-casinos_events.tsv.
The event "Stimulus" refers to multiple event values. Creating hierarchical event names.
    Renaming event: Stimulus -> Stimulus/S 21
    Renaming event: Stimulus -> Stimulus/S 22
    Renaming event: Stimulus -> Stimulus/S 23
    Renaming event: Stimulus -> Stimulus/S 24
    Renaming event: Stimulus -> Stimulus/S 27
    Renaming event: Stimulus -> Stimulus/S 21
    Renaming event: Stimulus -> Stimulus/S 22
    Renaming event: Stimulus -> Stimulus/S 23
    Renaming event: Stimulus -> Stimulus/S 24
    Renaming event: Stimulus -> Stimulus/S 26
    Renaming event: Stimulus -> Stimulus/S 21
    Renaming event: Stimulus -> Stimulus/S 22
    Renaming event: Stimulus -> Stimulus/S 23
    Renaming event: Stimulus -> Stimulus/S 24
    Renaming event: Stimulus -> Stimulus/S 26
  

Measurement date,"April 13, 2022 12:20:59 GMT"
Experimenter,Unknown
Participant,sub-36
Digitized points,Not available
Good channels,31 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.00 Hz
Lowpass,280.00 Hz


In [38]:
print(raw.info["bads"])

[]


In [39]:
raw.pick(["eeg", "eog"]).load_data()
raw.info

Measurement date,"April 13, 2022 10:06:01 GMT"
Experimenter,Unknown
Participant,sub-35
Digitized points,Not available
Good channels,31 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.00 Hz
Lowpass,280.00 Hz


In [ ]:
# print(raw.ch_names)

In [ ]:
# hacky try to replicate pipelines "virtual EOG channels" with FP1, FP2
# raw.set_channel_types({'Fp1':'eog', 'Fp2':'eog'})

# undo setting to eog
raw.set_channel_types({'Fp1':'eeg', 'Fp2':'eeg'})

Measurement date,"April 04, 2022 15:45:52 GMT"
Experimenter,Unknown
Participant,sub-30
Digitized points,Not available
Good channels,31 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.00 Hz
Lowpass,280.00 Hz


In [13]:
# testing other method which should ask to save changes
# throws error because "status" column is missing or smth?
# mne_bids.inspect_dataset(bids_path)

In [24]:
# picks = mne.pick_channels(raw.ch_names, include=[])
# raw.plot(order=picks, n_channels=len(picks), duration=70, lowpass=50, highpass=1, scalings={"eeg":100e-6})
# ;

In [3]:
# add bad channels to list

# raw.info["bads"].append("EEG 050")  # add a single channel
# raw.info["bads"].extend(["EEG 051", "EEG 052"])  # add a list of channels
# bad_chan = raw.info["bads"].pop(-1)  # remove the last entry in the list
# raw.info["bads"] = original_bads  # change the whole list at once

original_bads = deepcopy(raw.info["bads"])

# testing
# raw.info["bads"].extend([
#     "F7", 
#     "FC1"
# ])


In [43]:
# interactively selecting bad channels
mne.viz.plot_raw(raw, duration=70, lowpass=50, highpass=1, scalings={"eeg":100e-6},n_channels=len(raw.ch_names))
# mne.viz.plot_raw(raw, duration=15,n_channels=len(raw.ch_names))
;

Setting up band-pass filter from 1 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 50.00 Hz: -6.02, -6.02 dB



''

Channels marked as bad:
none
Channels marked as bad:
none


In [ ]:
print(raw.info["bads"])

['Fp1']


In [ ]:
# default is exclude='bads':
good_eeg = mne.pick_types(raw.info, meg=False, eeg=True)
all_eeg = mne.pick_types(raw.info, meg=False, eeg=True, exclude=[])
print(np.setdiff1d(all_eeg, good_eeg))
print(np.array(raw.ch_names)[np.setdiff1d(all_eeg, good_eeg)])

[]
[]


In [ ]:
# mne.viz.plot_raw(raw)
# ;